# Capstone Coding: Part 1 Converting Picture-based Data into Readable DataFrame.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import ollama
import os
import sys
import json
import random
import concurrent.futures
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from io import StringIO

In [ ]:
# Load the Excel file
relative_path = "list.xlsx"
data = pd.read_excel(relative_path, sheet_name = "Sheet1")
# Convert DataFrame to CSV string in memory
csv_buffer = StringIO()
data.to_csv(csv_buffer, index = False)
csv_buffer.seek(0)
# Read the CSV string into a DataFrame and display it
data = pd.read_csv(csv_buffer)
data['PhotoNumber'] = data.index+1
data['PhotoNumber'] = data['PhotoNumber'].astype(str)
data

In [ ]:
print(data.head())

In [ ]:
# Clean and standardize the 'final rank' column in-place
data['final rank'] = (
    data['final rank']
    .dropna()
    .astype(str)
    .str.strip()
    .str.replace('\t', '', regex=False)
    .str.title()
)

# Clean and standardize the 'Award' column in-place
# Step 1: Work only on non-null entries
data['Award'] = data['Award'].astype(str)

# Step 2: Replace non-standard separators and remove tabs/newlines/extra commas
data['Award'] = (
    data['Award']
    .str.replace('\t', '', regex=False)
    .str.replace('，', ',', regex=False)  # Chinese comma
    .str.replace(';', ',', regex=False)
    .str.replace('\n', ',', regex=False)
    .str.replace('\r', '', regex=False)
    .str.replace(',+', ',', regex=True)  # remove repeated commas
    .str.strip(', \t\n')  # trim leading/trailing commas or whitespace
)

# Step 3: Standardize capitalization for each individual award (optional, but powerful)
def title_case_awards(entry):
    if pd.isna(entry) or entry.strip() == '':
        return None
    awards = [award.strip().title() for award in entry.split(',') if award.strip()]
    return ', '.join(awards)

data['Award'] = data['Award'].apply(title_case_awards)

print(data.head())


In [ ]:
# Extract and clean the 'final rank' column
general_series = data['final rank'].dropna().astype(str)

# Drop duplicates and sort
unique_rank = general_series.drop_duplicates().sort_values().reset_index(drop=True)

# Display
print("✅ Cleaned Unique Ranks ({} total):".format(len(unique_rank)))
print(unique_rank.to_list())

In [ ]:
# Step 1: Load and clean the Award column
award_series = data['Award'].dropna().astype(str).str.lower()

# Step w: Split by comma, strip whitespace, explode to flatten
all_awards = award_series.str.split(',').explode().str.strip()

# Step 3: Drop empty entries and duplicates
clean_awards = all_awards[all_awards != ''].drop_duplicates().sort_values().reset_index(drop=True)

# Display result
print("✅ Cleaned Unique Awards ({} total):".format(len(clean_awards)))
clean_awards.to_list()
print(clean_awards)
# Save the cleaned unique awards to a text file for reference.
data_unique_award = clean_awards.to_csv('data_unique_rank.txt',index = False, header = False)

In [ ]:
# Mapping of award name variations to standardized names
award_mapping = {
    # Silver Star and variants
    'silver star medal': 'Silver Star',
    'silver citation star': 'Silver Star',
    'silver star citation': 'Silver Star',
    'silver star commendation': 'Silver Star',
    'silver star': 'Silver Star',

    # Purple Heart and variants
    'purple heart medal': 'Purple Heart',
    'purple heart': 'Purple Heart',

    # Distinguished Service Medals
    'distinguished service medal': 'Distinguished Service Medal',
    'army distinguished service medal': 'Distinguished Service Medal',
    'navy distinguished service medal': 'Distinguished Service Medal',
    'u.s. army distinguished service medal': 'Distinguished Service Medal',
    'distinguished service medal. purple heart': 'Distinguished Service Medal, Purple Heart',

    # Medal of Honor and variants
    'medal of honor': 'Medal of Honor',
    'congressional air force medal of recognition': 'Medal of Honor',

    # Bronze Star
    'bronze star medal': 'Bronze Star',
    'bronze star': 'Bronze Star',

    # Legion of Merit
    'legion of merit': 'Legion of Merit',

    # Legion of Honor and variants
    'legion of honor': 'Legion of Honor',
    'legion of honour': 'Legion of Honor',
    'légion d\'honneur (france)': 'Legion of Honor',
    'french légion d\'honneur': 'Legion of Honor',
    'french legion of honor': 'Legion of Honor',
    'commandeurs of the légion d\'honneur': 'Legion of Honor',
    'commander of the legion of honor': 'Legion of Honor',
    'commander of the legion of honour': 'Legion of Honor',
    'officer of the legion of honour': 'Legion of Honor',
    'officier of the legion of honor (france)': 'Legion of Honor',
    'commander of the legion of': 'Legion of Honor',
    'commander of the legion of honor (france)': 'Legion of Honor',
    'commander of the legion of honour (france)': 'Legion of Honor',
    'legion of honor (commander)': 'Legion of Honor',
    'legion of honor (commander) (france)': 'Legion of Honor',
    'legion of honor (france)': 'Legion of Honor',
    'legion of honor (officer) (france)': 'Legion of Honor',
    'legion of honour (france)': 'Legion of Honor',
    'legion of honour (officer) (france)': 'Legion of Honor',
    'honour (france)': 'Legion of Honor',

    # Croix de Guerre and variants
    'croix de guerre': 'Croix de Guerre',
    'croix de guerre with palm': 'Croix de Guerre',
    'croix de guerre (france)': 'Croix de Guerre',
    'croix de guerre with palm (france)': 'Croix de Guerre',
    'belgian croix de guerre': 'Croix de Guerre',

    # Orders and Medals
    'order of the crown (belgium)': 'Order of the Crown',
    'order of the crown of italy': 'Order of the Crown',
    'order of the crown': 'Order of the Crown',
    'order of the crown (italy)': 'Order of the Crown',
    'order of the crown of italy (commander)': 'Order of the Crown',
    'order of the crown of romania': 'Order of the Crown',
    'italian order of the crown': 'Order of the Crown',
    'belgian order of the crown': 'Order of the Crown',
    'commander of the order of the crown (belgium)': 'Order of the Crown',
    'commander of the order of the crown of italy': 'Order of the Crown',

    'order of leopold': 'Order of Leopold',
    'order of leopold (belgium)': 'Order of Leopold',
    'order of leopold (commander).': 'Order of Leopold',
    'order of leopold ii': 'Order of Leopold',
    'order of leopold ii (commander) (belgium)': 'Order of Leopold',
    'order of léopold (belgium)': 'Order of Leopold',
    'commander of the order of leopold': 'Order of Leopold',

    'spanish campaign medal': 'Spanish Campaign Medal',
    'spanish war service medal': 'Spanish War Service Medal',
    'philippine campaign medal': 'Philippine Campaign Medal',
    'philippine congressional medal': 'Philippine Congressional Medal',
    'civil war campaign medal': 'Civil War Campaign Medal',
    'indian campaign medal': 'Indian Campaign Medal',
    'mexican border service medal': 'Mexican Border Service Medal',
    'cuban pacification medal': 'Cuban Pacification Medal',
    'world war i victory medal': 'World War I Victory Medal',
    'the victory medal': 'World War I Victory Medal',

    # Order of the Bath
    'order of the bath': 'Order of the Bath',
    'order of the bath (united kingdom)': 'Order of the Bath',
    'companion of the order of the bath': 'Order of the Bath',
    'companion of the bath': 'Order of the Bath',
    'commander of the order of the bath': 'Order of the Bath',

    # Order of St. Michael and St. George
    'order of st michael and st george': 'Order of St Michael and St George',
    'order of st. michael & st. george': 'Order of St Michael and St George',
    'order of st. michael and st. george (britain)': 'Order of St Michael and St George',
    'order of st michael and st george (companion) (great britain)': 'Order of St Michael and St George',

    # Other Orders
    'order of saints maurice and lazarus': 'Order of Saints Maurice and Lazarus',
    'order of saints maurice and lazarus (commander) (italy)': 'Order of Saints Maurice and Lazarus',
    'order of the dragon of annam (commander) (french indochina)': 'Order of the Dragon of Annam',
    'order of the rising sun': 'Order of the Rising Sun',
    'order of the star of africa (liberia)': 'Order of the Star of Africa',
    'order of wen hu': 'Order of Wen Hu',
    'order of wen-hu': 'Order of Wen Hu',
    'order of la solidaridad': 'Order of La Solidaridad (Panama)',

    # Others
    'oak leaf cluster': 'Oak Leaf Cluster',
    'medal of military merit (greece) (first class)': 'Medal of Military Merit',
    'public welfare medal': 'Public Welfare Medal',
    'navy cross': 'Navy Cross',
    'distinguished service cross': 'Distinguished Service Cross',
    'war merit cross': 'War Merit Cross',
    'czechoslovak war cross': 'Czechoslovak War Cross 1918',
    'grand officer of the order of the sacred treasure': 'Grand Cordon of the Order of the Sacred Treasure',
    'honorary knight commander': 'Honorary Knight Commande',
    'citation star': 'Citation Star'
}

In [ ]:
# Write a function to map and split each 'Award' entry into a standardized list
def map_and_split_awards(entry):
    if pd.isna(entry):
        return []
    normalized = (
        entry.replace('\n', ',')
             .replace('\t', '')
             .replace(';', ',')
             .lower()
    )
    items = [itm.strip() for itm in normalized.split(',') if itm.strip()]
    # Map each item to its standardized form, defaulting to title-cased original
    return list({award_mapping.get(itm, itm.title()) for itm in items})

# Apply to your DataFrame
data['award_list'] = data['Award'].apply(map_and_split_awards)

# Explode and create dummy variables
exploded = data['award_list'].explode()
dummies = (
    pd.get_dummies(exploded)
      .groupby(level=0)
      .max()
      .astype(int)
)

# Concatenate the dummy columns back into the original DataFrame
data = pd.concat([data, dummies], axis=1)

# Now, `data` contains one 0/1 column for each standardized award.
data.head()


In [ ]:
# Drop the rows if rank is one of the unwanted ones
drop_ranks = ['Colonel', 'Lieutenant Colonel', 'Private', 'Private 2']
data = data[~data['final rank'].isin(drop_ranks)]

# Drop the Award column entirely
data = data.drop(columns=['Award'])
data = data.drop(columns = ['award_list'])
# Extract the start and end years (handles “‑” vs “–” etc.)
# This pulls out the first and second 4‑digit sequences in the string.
years = data['years of service'].astype(str).str.extract(r'(\d{4}).*?(\d{4})')
data['beginning'] = pd.to_numeric(years[0], errors='coerce')
data['end'] = pd.to_numeric(years[1], errors='coerce')

# Compute service length and clean up
data['Number of Years Served'] = data['end'] - data['beginning']
data = data.drop(columns=['years of service', 'beginning', 'end'])
data.dropna(subset = ['Number of Years Served'])

# nspect the head
data.head()


In [ ]:
def get_image_path(directory):
    valid_extansions = ['.jpg', '.jpeg', '.png', '.bmp']
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.lower().endswith(tuple(valid_extansions))]

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
caption_model = BlipForConditionalGeneration.from_pretrained('Salesforce/blip-image-captioning-base')

def caption_image(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f'Error opening image {image_path}:{e}')
        return ""
    inputs = processor(images = image, return_tensors = 'pt')
    output_ids = caption_model.generate(**inputs, max_length = 60)
    caption = processor.decode(output_ids[0], ski_special_tokens = True)
    return processor.decode(output_ids[0], skip_special_tokens = True).strip()


def extract_facial_features(image_path):
    caption = caption_image(image_path)
    if not caption:
        return None
    prompt = f"""
You are an expert in facial dominance analysis of U.S. military portraits.
Rate how dominant or submissive the person appears on this 7-point scale:
1 = Very Submissive
2 = Moderately Submissive
3 = Slightly Submissive
4 = Neutral
5 = Slightly Dominant
6 = Moderately Dominant
7 = Very Dominant

Respond ONLY with valid JSON containing:
  \"rating\": integer between 1 and 7,
  \"explanation\": one concise sentence (≤ 50 words).

Portrait caption: {caption}
"""
    try:
        response = ollama.generate(model = 'llama3', prompt = prompt)
        raw = response.get("response", "")
        print("🔍 Raw LLM output:\n", raw)
        data = json.loads(response.get("response", "{}"))
        data['image'] = os.path.basename(image_path)
        data['caption'] = caption
        return data
    except Exception as e:
        print(f"❌ LLM failed for {image_path}: {e}")
        return None

def build_dominance_dataframe(directory):
    records = []
    for path in get_image_path(directory):
        out = extract_facial_features(path)
        if out:
            records.append(out)
    if not records:
        return pd.DataFrame(columns = ['image', 'caption', 'rating', 'explanation'])
    
    new_data = pd.DataFrame(records)
    new_data['rating'] = new_data['rating'].astype(int)
    new_data['explanation'] = new_data['explanation'].astype(str)
    return new_data[['image', 'caption', 'rating', 'explanation']]




In [ ]:
path = "Photos"
new_data = build_dominance_dataframe(path)
new_data

In [ ]:
data.set_index('PhotoNumber', inplace = True)
data

In [ ]:
merged_data = pd.merge(data, new_data, left_index = True, right_index = True, how = 'left')
merged_data

In [ ]:
merged_data = merged_data.dropna(subset = ['rating'])
merged_data = merged_data.drop(columns = ['Black', '(Commander)', '(Grand Officer)', 'image', 'PhotoNumber'])
merged_data

In [ ]:
merged_data = merged_data.to_csv('merged_data.csv', index = False, header = True)